# Clean Data
## Introduction
To measure the quality of the air we can use Sulfur Monoxide (SO), Nitrogen Dioxide (NO2), Carbon Dioxide (CO2), Ozone (O3) or Total Suspended Particules. The data obtained by OpenAQ is provided by the different sensors around the world. Nevertheless, we will focus on our country during the last year. We consider using 2020 because it is a year that we have a pandemic situation and we would know what happens during the lockdown etc.

## Read Files

In [4]:
import pandas as pd
import os
import json
from pandas_profiling import ProfileReport
from IPython.display import display

rel_path="../data/raw/"
clean_path="../data/clean/"

parameters = json.load(open(rel_path +'parameters.json'))
parameters = pd.DataFrame(parameters['results']).set_index('id')

profile = ProfileReport(parameters, title="Profiling Parameters")
profile.to_widgets()

print(parameters.info(),'\n')

parameters = parameters[['displayName','name','preferredUnit']]

display(parameters)

parameters.to_json(clean_path+"parameters.json",orient='records')


<class 'pandas.core.frame.DataFrame'>
Int64Index: 22 entries, 1 to 19843
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   description    22 non-null     object 
 1   displayName    22 non-null     object 
 2   isCore         22 non-null     bool   
 3   maxColorValue  11 non-null     float64
 4   name           22 non-null     object 
 5   preferredUnit  22 non-null     object 
dtypes: bool(1), float64(1), object(4)
memory usage: 1.1+ KB
None 



,displayName,name,preferredUnit
id,,,
1,PM10,pm10,µg/m³
2,PM2.5,pm25,µg/m³
3,O₃ mass,o3,µg/m³
4,CO mass,co,µg/m³
5,NO₂ mass,no2,µg/m³
6,SO₂ mass,so2,µg/m³
7,NO₂,no2,ppm
8,CO,co,ppm
9,SO₂,so2,ppm


In [2]:
countries = json.load(open(rel_path +'countries.json'))
countries = pd.DataFrame(countries['results'])

display(countries)

countries = countries[['cities','code','name','parameters']]

spain = countries[countries['name']=='Spain']

display(spain)

print(spain['parameters'].values[0])

countries.to_json(clean_path+"countries.json",orient='records')

,cities,code,count,firstUpdated,lastUpdated,locations,name,parameters,sources
0,2,AD,93301027,2017-09-14T23:00:00+00:00,2021-02-01T12:00:00+00:00,3,Andorra,"[co, no2, o3, pm10, so2]",1
1,3,AE,5764752629,2017-12-26T23:00:00+00:00,2021-02-01T13:29:47+00:00,5,United Arab Emirates,"[o3, pm1, pm10, pm25, um010, um025, um100]",5
2,2,AF,51956121,2019-10-20T22:30:00+00:00,2021-02-01T11:30:00+00:00,2,Afghanistan,[pm25],3
3,0,AM,7459800876,2021-01-21T00:00:06+00:00,2021-02-01T13:29:16+00:00,2,Armenia,"[pm1, pm10, pm25, um010, um025, um100]",1
4,0,AO,3842638218,2021-01-21T00:02:35+00:00,2021-02-01T13:29:42+00:00,2,Angola,"[pm1, pm10, pm25, um010, um025, um100]",1
...,...,...,...,...,...,...,...,...,...
125,1,UZ,66948148,2018-11-27T22:00:00+00:00,2021-02-01T11:00:00+00:00,1,Uzbekistan,"[o3, pm25]",3
126,1,VM,67509765,2016-11-09T17:00:00+00:00,2021-02-01T11:00:00+00:00,2,VM,[pm25],4
127,2,VN,22144692164,2015-05-04T02:00:00+00:00,2021-02-01T13:29:43+00:00,9,Vietnam,"[pm1, pm10, pm25, um010, um025, um100]",3
128,8,XK,4838155523,2016-04-08T20:00:00+00:00,2021-02-01T13:28:28+00:00,16,Kosovo,"[co, no2, o3, pm1, pm10, pm25, so2, um010, um0...",4


,cities,code,name,parameters
38,59,ES,Spain,"[co, no2, o3, pm1, pm10, pm25, so2, um010, um0..."


['co', 'no2', 'o3', 'pm1', 'pm10', 'pm25', 'so2', 'um010', 'um025', 'um100']


In [9]:
measurements = json.load(open(rel_path +'measurements_day_0.json'))
measurements = pd.DataFrame(measurements['results'])

measurements = measurements[['average','day','parameter','unit']]
pd.to_datetime(measurements['day'],format='%Y-%m-%d')

print(measurements.info())

measurements.to_json(clean_path+"measurements.json",orient='records')
display(measurements)

profile = ProfileReport(measurements, title="Profiling Measurements")
profile.to_widgets()
#measurements.loc['2020-12-31','o3']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2182 entries, 0 to 2181
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   average    2182 non-null   float64
 1   day        2182 non-null   object 
 2   parameter  2182 non-null   object 
 3   unit       2182 non-null   object 
dtypes: float64(1), object(3)
memory usage: 68.3+ KB
None


,average,day,parameter,unit
0,52.0092,2020-12-31,o3,µg/m³
1,11.5678,2020-12-31,pm10,µg/m³
2,14.7604,2020-12-31,no2,µg/m³
3,3.4066,2020-12-31,so2,µg/m³
4,322.8080,2020-12-31,co,µg/m³
...,...,...,...,...
2177,3.1374,2020-01-01,so2,µg/m³
2178,25.6587,2020-01-01,pm10,µg/m³
2179,412.7953,2020-01-01,co,µg/m³
2180,22.7752,2020-01-01,no2,µg/m³


NameError: name 'ProfileReport' is not defined